# Register Custom Monitor in OpenScale
This notebook 
1. registers the custom metric provider as an integrated system in openscale
2. creates the corresponding monitor definition
3. updates monitor metadata

In [ ]:
from ibm_watson_openscale import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *
import urllib3
urllib3.disable_warnings()

from datetime import datetime, timezone, timedelta
import uuid

import os
import wos_sdk_utils as wos_util

In [ ]:
scoring_url = 'https://internal-nginx-svc:12443/ml/v4/deployments/7ae28492-bbd3-404c-8b3d-ba9181f91f54/predictions?version=2022-04-01'
openscale_integrated_system_name = "custom dummy metrics wendy"
openscale_monitor_name = 'dummy monitor wendy'
openscale_monitor_id = None

credentials = {'username': 'wangw6',
               'api_key': 'kkdAZrXOmFdo3T3d94wFcHW2ith01aP9V4XL1NmN'}

## 1. Register Scoring Endpoint in OpenScale

In [ ]:
wos_client = wos_util.get_client()
wos_client.version

In [ ]:
# Delete existing custom metrics provider integrated systems if present
wos_util.integrated_system_delete(openscale_integrated_system_name,wos_client)

In [ ]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=openscale_integrated_system_name,
    description=openscale_integrated_system_name,
    type="custom_metrics_provider",
    credentials= {"auth_type":"bearer",
                  "token_info": {
                      "url": "{}/icp4d-api/v1/authorize".format(os.environ['RUNTIME_ENV_APSX_URL']),
                      "headers": {"Content-Type": "application/json",
                                  "Accept": "application/json"},
                      "payload": {'username':credentials['username'],
                                   'api_key':credentials['api_key']},
                      "method": "post"}
                 },
    connection={"display_name": openscale_integrated_system_name,
                "endpoint": scoring_url
    }).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(integrated_system_id)

## 2. Setup Custom Monitor Definition

In [ ]:
openscale_monitor_defaults = {'specificity': {'threshold':[0.5,'lower']},
                              'sensitivity': {'threshold':[0.6,'lower']}
                             }

In [ ]:
monitor_id = wos_util.monitor_definition_create(openscale_monitor_name,
                                                openscale_monitor_defaults,
                                                wos_client,overwrite=True)
print(monitor_id)

In [ ]:
wos_client.monitor_definitions.show()

In [ ]:
wos_util.get_monitor_definition(wos_client,monitor_name=openscale_monitor_name)